# Work FLow
    1.import Libraries
    2.input embedding
    3.Positional Encoding
    4.Multi Head Attention(Self Attention)
    5.Add & Normalized
    6.Feed Forward
    7.Residual Connection
    8.Encoder
    9.Decoder
    10.Build a Transformer
    11.Testing Transformer
    12.Tokenizer
    13.Loading Dataset
    14.Valid Loop
    15.Traning Loop
    16.Conclusion

## 1.import Libraries

In [8]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,random_split
from torch.utils.tensorboard import SummaryWriter

import math

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

from pathlib import Path

from typing import Any

from tqdm import tqdm

# Input Embedding

In [9]:
class InputEmbedding(nn.Module):
    def __init__(self,d_model:int,vocab_size:int):
        super().__init__()
        self.d_model=d_model
        self.vocab_size = vocab_size
        self.embedding=nn.Embedding(vocab_size,d_model)
        
    def forward(self,x):
        return self.embedding(x)*math.sqrt(self.d_model)

# Position Encoding 

In [11]:
class PositionEncoding(nn.Module): 
    def __init__(self,d_model:int,seq_len:int,dropout:float):
        super().__init__()
        self.d_model=d_model
        self.seq_len=seq_len
        self.dropout=nn.Dropout(dropout)
        
        pe=torch.zero(seq_len,d_model)
        positon =torch.arange(0,seq_len,dtype=float).unsqueeze(1)# shape (seq_len,1)
        div_term=torch.exp(torch.arange(0,d_model,2).float() * (-math.log(10000.0)/d_model))
        pe[:,0::2]=torch.sin(positon*div_term)
        pe[:,1::2]=torch.cos(positon*div_term)
        pe=pe.unsqueeze(0)
        self.register_buffer("pe",pe)
        
    def forward(self,x):
        x=x+(self.pe[:,:x.shape[1],:]).requires_grad_(False)
        return self.dropout(x)